In [1]:
dataset = "IN-Abs" # Options: IN - IN-Abs, UK-UK-Abs, N2-IN-Ext
output_path = "/kaggle/working/output"
import pandas as pd
import numpy as np
import glob
import sys
sys.path.insert(0, '../')

In [2]:
cd /kaggle/input/utilitis

/kaggle/input/utilitis


In [3]:
from utilities import *
import os
import nltk
if not os.path.exists(output_path):
    os.makedirs(output_path)
#Reading the train documents
names, data_source, data_summary = get_summary_data(dataset, "train")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
import pandas as pd
#cleaning the data
def cleanData(data_source,data_summary):
    cleaned_data_source=list()
    cleaned_data_summary=list()

    for data in data_source:
        cleaned_text = data.replace('\n', ' ').replace('\t', ' ').replace('\\' ,' ')
        cleaned_data_source.append(cleaned_text)

    for data in data_summary:
        cleaned_text = data.replace('\n', ' ').replace('\t', ' ')
        cleaned_data_summary.append(cleaned_text)

    df = pd.DataFrame(list(zip(cleaned_data_source,cleaned_data_summary)),columns =['data', 'summary'])
    df.rename(columns = {'data':'source', 'summary':'target'}, inplace = True)
    
    return df


In [5]:
df=cleanData(data_source,data_summary)

In [6]:
df

,source,target
0,iminal Appeal No. 47 of 1963. Appeal by specia...,"The appellant published in his paper, which ha..."
1,minal Appeal No. 210 of 1963. Appeal by specia...,On the first information report lodged by the ...
2,Civil Appeal No. 2618 of 1983. From the Judgme...,The first respondent who lost to the appellant...
3,ivil Appeal No. 2991 of 1986. From the Judgmen...,The appellant was sought to be evicted under s...
4,Civil Appeal No. 1733 of 1973. From the Judgme...,A firm (JTC) obtained the sole selling agency ...
...,...,...
7025,Appeals Nos. 98 and 99 of 1957. Appeals from t...,In order to put the sugar industry on a stable...
7026,Civil Appeal No. 1931 of 1969. Appeal by Speci...,In the eviction suit filed by the respondents ...
7027,Appeal No. 81 of 1990. From the Judgment and O...,Under Section 21(1)(a) of the Rajasthan Civil ...
7028,ivil Appeal No. 1993 of 1982. From the Judgmen...,The respondent was the daughter of the origina...


In [7]:
#test dataset
test_names, test_data_source, test_data_summary = get_summary_data(dataset, "test")
df_test= cleanData(test_data_source,test_data_summary)
df_test




,source,target
0,Appeal No. 101 of 1959. Appeal by special leav...,The appellants who are displaced persons from ...
1,it case for initiating any procee d ings for c...,The petitioners in the writ petitions were app...
2,Appeal No. 647 of 1966. Appeal by special leav...,The first respondent while he was holding the ...
3,l Misc. Petitions Nos. 401/and 4012 of 1968. A...,M/s Tarapore & Co. who were plaintiffs in a su...
4,N: Criminal Appeals Nos. 39 40 of 1976. (From ...,The appellants wrote a letter to the President...
...,...,...
95,: Civil Appeals Nos 188 to 190 of 1958. Appeal...,The principal question for determination in th...
96,Appeal No. 205 of 1954. On appeal from the jud...,"By section ,12, sub section 1, of the (XXXI of..."
97,ivil Appeal No. 196 of 1 970. From the judgmen...,The plaintiffs in their suit claimed that the ...
98,Civil Appeal Nos. 1617 and 1640A of 1979. Appe...,"On September 30, 1978 the Collector published ..."


In [8]:
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

# Assuming you have a DataFrame 'df' with columns 'text' and 'summary'
# Replace 'df' with your DataFrame containing the text and summary columns

# Load the T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base',model_max_length=512)
model = T5ForConditionalGeneration.from_pretrained('t5-base').to('cuda')  # Move the model to GPU

# Preprocess the DataFrame
def preprocess_dataframe(dataframe):
    inputs = [f"summarize:{text}" for text in dataframe['source']]
    targets = dataframe['target']
    
    return {'inputs': inputs, 'targets': targets}

# Prepare the training data
train_data = preprocess_dataframe(df)
test_data = preprocess_dataframe(df_test)





spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:

# Define the training arguments
training_args = TrainingArguments(
    output_dir='/kaggle/working/results',  # Output directory
    num_train_epochs=3,  # Number of training epochs
    per_device_train_batch_size=5,  # Batch size per device during training
    per_device_eval_batch_size=5,   # Batch size for evaluation
    logging_dir='/kaggle/working/logs',  # Directory for storing logs
    logging_steps=100,  # Log training progress every 100 steps
    save_steps=100,  # Save checkpoint every 1000 steps
    evaluation_strategy="steps",  # Evaluate every 'logging_steps'
    eval_steps=100,  # Evaluate every 1000 steps
    save_total_limit=3,  # Limit the total number of checkpoints
    gradient_accumulation_steps=16,  # Accumulate gradients to simulate larger batches
    fp16=True,  # Enable mixed precision training
    report_to='none',  # suppress reports for Kaggle notebook
)


In [10]:
from torch.utils.data import Dataset, DataLoader
import torch

# Define a custom PyTorch Dataset
class CustomDataset(Dataset):
    def __init__(self, inputs, targets, tokenizer):
        self.inputs = inputs
        self.targets = targets
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = self.inputs[idx]
        target_text = self.targets[idx]

        input_encoding = self.tokenizer.encode_plus(input_text, return_tensors='pt', padding='max_length', max_length=512, truncation=True)
        target_encoding = self.tokenizer.encode_plus(target_text, return_tensors='pt', padding='max_length', max_length=512, truncation=True)

        return {
            'input_ids': input_encoding['input_ids'].flatten(),
            'attention_mask': input_encoding['attention_mask'].flatten(),
            'labels': target_encoding['input_ids'].flatten(),
            'decoder_attention_mask': target_encoding['attention_mask'].flatten()
        }

# Convert data to PyTorch Dataset
train_dataset = CustomDataset(train_data['inputs'], train_data['targets'], tokenizer)
test_dataset =  CustomDataset(test_data['inputs'], test_data['targets'], tokenizer)

# DataLoader for batching
train_dataloader = DataLoader(train_dataset, batch_size=2)
test_dataloader = DataLoader(test_data, batch_size=2)


# # Define Trainer with DataLoader
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

# # Start the fine-tuning process

trainer.train()

# # Save the fine-tuned model
# model.save_pretrained('/kaggle/working/fine_tuned_model')


Step,Training Loss,Validation Loss
100,4.132600,2.905257
200,3.209100,2.800519


TrainOutput(global_step=261, training_loss=3.543160215648198, metrics={'train_runtime': 4302.7839, 'train_samples_per_second': 4.901, 'train_steps_per_second': 0.061, 'total_flos': 1.27150407548928e+16, 'train_loss': 3.543160215648198, 'epoch': 2.97})

In [11]:
# Save the fine-tuned model
# model.save_pretrained('/kaggle/working/fine_tuned_model')
torch.save(model.state_dict(),"/kaggle/working/T5Model.pt")